# Auditory Oddball Paradigm
#### Alexis Pomares Pastor <i>- May 2021</i>

In [1]:
import pygame
from pygame.locals import *
import numpy as np
import pandas as pd
import random
import time

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generate_sounds(n_beeps, n_runs, beep_duration, freq, ratio_oddball, p_id):
    dt = 1/44100
    beep_duration /= 1000
    t_simple = np.arange(dt, beep_duration, dt)
    fade = 0.01
    amplification_f = 16383
    ramp = np.linspace(0, 1, int(fade/dt))
    pad_l = 300
    sound_len = int(beep_duration/dt + pad_l - 1)
    
    s = np.zeros((len(freq), sound_len))
    for sind in range(s.shape[0]):
        sound_temp = np.sin(2*np.pi * freq[sind] * t_simple) * amplification_f
        sound_temp = np.multiply(sound_temp, np.concatenate([ramp, np.ones(len(t_simple) - 2*len(ramp)), ramp[::-1]]))
        s[sind] = np.concatenate([sound_temp, np.zeros(pad_l)])
    
    sounds = np.zeros((n_runs, n_beeps, sound_len))
    oddballs = np.zeros((n_runs, int(n_beeps * ratio_oddball)))
    try: random.seed(int(p_id))  #for reproducibility, set seed to Participant ID
    except: pass

    for r in range(n_runs):
        sounds_prep = np.zeros((n_beeps, sound_len))
        
        iind = np.sort(np.array(random.sample(range(0, n_beeps), int(n_beeps * ratio_oddball))))  #randomize oddball positions
        while ratio_oddball < 0.45 and (any(np.diff(iind) <= 1) or iind[0] < 2):               #ensure no consecutive oddballs
            iind = np.sort(np.array(random.sample(range(0, n_beeps), int(n_beeps * ratio_oddball))))
        
        sounds_prep[::] = s[0]
        sounds_prep[np.array(iind)] = s[1]  #replace the oddball sounds
        sounds[r] = sounds_prep
        
        oddballs[r][::] = iind              #keep track of oddball indexes
    
    oddballs = np.sum(np.eye(n_beeps)[oddballs.astype(int)], 1)   #one-hot representation, useful for performance assessment

    return [sounds, oddballs]

In [3]:
def start_with_countdown(screen, current_run):
    font = pygame.font.SysFont('Consolas', 100)    
    text = f'Press SPACE for run{current_run}...'.center(25)
    waiting = True
    
    while waiting:
        for e in pygame.event.get():
            if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE: 
                waiting = False
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return False
        screen.fill((0, 0, 0))
        screen.blit(font.render(text, True, (80, 80, 80)), (90, 380))
        pygame.display.flip()
    
    pygame.time.set_timer(pygame.USEREVENT, 1000)
    font = pygame.font.SysFont('Consolas', 300)    
    counter, text = 3, '3'.center(1)
    counting = True
    
    while counting:
        for e in pygame.event.get():
            if e.type == pygame.USEREVENT: 
                counter -= 1
                if counter > 0:
                    text = str(counter).center(1)
                else:
                    counting = False
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return False
        screen.fill((0, 0, 0))
        screen.blit(font.render(text, True, (255, 255, 255)), (680, 300))
        pygame.display.flip()
    
    return True

In [4]:
def end_with_cooldown(screen, is_last_run = False, cooldown = 5):
    screen.fill((0, 0, 0))
    if is_last_run:
        font = pygame.font.SysFont('Consolas', 80)
        screen.blit(font.render('Thanks for participating :)'.center(30), True, (200, 200, 200)), (100, 380))
    else:
        font = pygame.font.SysFont('Consolas', 100)
        screen.blit(font.render('Nice job!'.center(10), True, (200, 200, 200)), (550, 380))
    pygame.display.flip()

    start_time = pygame.time.get_ticks()
    end_time = pygame.time.get_ticks() + cooldown*1000

    while start_time < end_time:
        start_time = pygame.time.get_ticks()                
        for e in pygame.event.get():
            if e.type == pygame.KEYDOWN and e.key == pygame.K_RETURN:  #cooldown can be skiped by pressing ENTER
                start_time = end_time
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return True
    return False

In [5]:
def run_session(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, start_from_run = 0):
    pygame.mixer.pre_init(44100, -16, 2, 32)   #setup mixer to avoid sound lag
    pygame.init()
    pygame.display.set_caption("Oddball Session 🕹️")
    screen = pygame.display.set_mode([1536, 864])  #set up drawing window (leave empty for fullscreen mode)
    bg = pygame.image.load("background.jpg")       #artsy background

    for r in range(start_from_run, n_runs):
        print("Starting run #{}! 🏁      Time: {}".format(r, time.strftime("%H:%M:%S")))
        sounds_run = np.array(sounds[r])

        playing = start_with_countdown(screen, r)

        last_time = time.time()
        b = 0

        while b <= n_beeps and playing:
            screen.blit(bg, (0,0))
            pygame.display.flip()
            
            if (time.time() - last_time) > (silence_duration/1000 - 0.005):  #play sound
                if b < n_beeps:
                    pygame.mixer.Sound(sounds_run[b, :].astype('int16')).play()
                    timings[r, b, 0] = pygame.time.get_ticks()/1000               #record when sound plays
                    
                last_time = time.time()
                b += 1
            
            for e in pygame.event.get():
                if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                    print("🛑 Script interrupted by user")
                    playing = False
                if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE and b > 0:   #record when user reacts
                    timings[r, b-1, 1] = pygame.time.get_ticks()/1000 - timings[r, b-1, 0]
        
        if not playing or end_with_cooldown(screen, r == n_runs-1, cooldown = 20): break  #wait until next run can start, or press ENTER to skip

    pygame.quit()  #exit application
    
    results = pd.DataFrame("", index = ["Run " + str(i) for i in range(n_runs)], columns = range(n_beeps))  #spreadsheet with readable results
    for i in range(start_from_run, r+1):
        for j in range(n_beeps):
            if oddballs[i, j] == 0:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "-"
                else:
                    results.at["Run " + str(i), j] = "Wrong"
            else:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "Missed"
                else:
                    results.at["Run " + str(i), j] = np.round(timings[i, j, 1], 3)
    
    return results

In [9]:
#Run this block once, to setup the parameters for the whole experimental session:

n_beeps = 30              #per run
n_runs = 10               #per whole experimental session
beep_duration = 50        #in ms
silence_duration = 1000   #between beeps, in ms
standard_freq = 400       #normal sound frequency, in Hz
oddball_freq = 1000       #odd sound frequency, in Hz
ratio_oddball = 0.2       #proportion of oddball/standard sounds

results_path = "results/"  #output folder (not filename) to save results

print("🔊 Remember to turn up computer volume to max\n")
p_id = input('Participant ID: ')

timings = np.zeros((n_runs, n_beeps, 2))  #initialize array to save times when sound played and when participant reacted
sounds, oddballs = generate_sounds(n_beeps, n_runs, beep_duration, [standard_freq, oddball_freq], ratio_oddball, p_id)

🔊 Remember to turn up computer volume to max



Participant ID:  P000


In [10]:
#If the session needs to be stopped, re-run this block with an adjusted 'start_from_run' value:
results = run_session(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, start_from_run = 0)

Starting run #0! 🏁      Time: 17:01:44
Starting run #1! 🏁      Time: 17:45:38
Starting run #2! 🏁      Time: 17:51:46
Starting run #3! 🏁      Time: 17:57:42
Starting run #4! 🏁      Time: 18:03:11
Starting run #5! 🏁      Time: 18:11:52
Starting run #6! 🏁      Time: 18:18:31
Starting run #7! 🏁      Time: 18:30:39
Starting run #8! 🏁      Time: 18:36:36
Starting run #9! 🏁      Time: 18:43:30


In [11]:
#Save results to an external ouput file:
filename = results_path + f"{p_id}_{time.strftime('%y%m%d_%H%M%S')}.csv"
results.to_csv(filename)    #save results to a CSV file
results                     #print results for visualization

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
Run 0,-,-,-,-,0.336,-,-,-,0.27,-,...,-,-,0.293,-,-,0.468,-,-,-,0.415
Run 1,-,Wrong,-,-,-,-,-,-,-,-,...,0.29,-,-,-,0.293,-,-,0.345,-,-
Run 2,Wrong,-,-,0.262,-,-,0.271,-,-,-,...,-,Wrong,-,-,-,-,-,0.236,-,-
Run 3,-,-,-,-,-,0.248,-,-,Missed,-,...,-,0.288,-,-,-,-,0.242,-,-,-
Run 4,-,-,-,-,0.269,-,0.339,-,-,-,...,-,-,0.329,-,-,-,-,-,-,0.265
Run 5,Wrong,-,-,-,0.31,-,0.285,-,-,-,...,-,0.248,-,-,-,-,-,-,-,-
Run 6,-,-,-,-,-,-,-,0.266,-,0.305,...,-,-,-,-,0.272,-,0.301,-,-,-
Run 7,-,-,-,-,0.3,-,0.3,-,-,-,...,-,0.291,-,0.341,-,-,-,-,-,-
Run 8,-,-,0.295,-,-,-,-,-,0.275,-,...,-,-,-,-,-,0.267,-,0.458,-,-
Run 9,-,-,-,-,-,0.254,-,-,-,0.279,...,0.376,-,-,-,-,0.304,-,-,-,-
